## Imports

In [2]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook"


from TDSE_solve import TDSE
from Slater_State import Slater
from Molecule import get_molecule

from random import randint, seed


/Users/milaimkas/Teaching/Teaching_project/venv/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning:

Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.



## Classes and functions (wrapper)

In [19]:
class Build_basis:
   def __init__(self, molecule_name="CO", basis=0):

      # Choose molecular system
      self.mol,self.mf = get_molecule(molecule_name,0)

      # build Slater states
      S_State = Slater(self.mol, self.mf)  # slater state object
      S_State.build()

      det = S_State.det  # Slater determinants
      self.dim = len(det)
      self.Ene = S_State.Ene  # energie of the Slater states
      self.DE = S_State.DE  # list of Delta(E)
      self.S_tdm = S_State.S_tdm  # transition dipole elements
      self.Orb_ex_ind = np.asarray(S_State.Orb_ex_ind) # index of orbital transition

      self.HOMO_idx = np.argmax(np.where(S_State.mo_ene > 0, S_State.mo_ene, 0))

      self.Result = None 
      self.ct_list = None
      self.t_list = None
      self.V_list = None 
      self.laser = None
        
   def Solve_TDSE(self,laser,tf,dt=0.005):
    
      # Expansion coefficients, initial condition
      c0 = np.zeros(self.dim,dtype=complex)
      c0[0] = complex(1.0,1.0)

      # Solve TDSE
      my_tdse = TDSE(self.S_tdm,self.DE,laser=laser)
      Result = my_tdse.Solve(c0, tf,dt=dt)
      self.ct_list, self.t_list, self.V_list = Result
      self.laser = laser
     
   def _Plot(self,thres):
        
      # colors
      seed(6) # for nice colors
      colors = []
      for i in range(self.dim):
         colors.append('#%06X' % randint(0, 0xFFFFFF))
         color_field = ('black','red','blue')

      # initialize figure
      fig = plt.figure()
      ax = fig.add_axes([0.1, 0.1, 1, 1.15])
      ax.set_xlabel("time (au)")
      ax.set_ylabel("$|c_i(t)|^2$")
      # plot each probability amplitude (|c^2|) as a function of time
      for i in range(self.dim):
         if np.max(self.ct_list[:,i]) > thres:
            if (self.Orb_ex_ind[i,0] == 0 ) & (self.Orb_ex_ind[i,1] == 0):
               ax.plot(self.t_list,self.ct_list[:,i], 'k--', label=f"{self.Orb_ex_ind[i,0]}->{self.Orb_ex_ind[i,1]}")
            else:
               ax.plot(self.t_list,self.ct_list[:,i],label=f"{self.Orb_ex_ind[i,0]}->{self.Orb_ex_ind[i,1]}",color=colors[i])

      # add subplot for field      
      inset = fig.add_axes([0.8, 0.5, 0.25, 0.30]) # left, bottom, width, height
      inset.set_title("Field")
      inset.set_xlabel("time (au)")

      for i in range(self.V_list[0,:].shape[0]):
         inset.plot(self.t_list,self.V_list[:,i],color=color_field[i])

      ax.legend(bbox_to_anchor=(1.05, 1), ncol=6, shadow=True, title="Legend", fancybox=True,loc=2, borderaxespad=0.)
      
      return fig
   
   def Plot(self,thres):
        
      # colors
      seed(6) # for nice colors
      colors = []
      for i in range(self.dim):
         colors.append('#%06X' % randint(0, 0xFFFFFF))
         color_field = ('black','red','blue')

      # initialize figure
      #fig = go.Figure()
      fig = make_subplots(
         rows=2, cols=2,
         specs=[[{"rowspan": 2},None],
               [None,{}]],
         print_grid=True)

      # plot each probability amplitude (|c^2|) as a function of time
      for i in range(self.dim):
         if np.max(self.ct_list[:,i]) > thres:
            if (self.Orb_ex_ind[i,0] == 0 ) & (self.Orb_ex_ind[i,1] == 0):
               fig.add_trace(go.Scatter(x=self.t_list, y=self.ct_list[:,i],
                    mode='lines',
                    name=f"{self.Orb_ex_ind[i,0]}->{self.Orb_ex_ind[i,1]}",
                    line=dict(color=colors[i], width=2, dash='dash')),
                    row=1, col=1)
            else:
               fig.add_trace(go.Scatter(x=self.t_list, y=self.ct_list[:,i],
                    mode='lines',
                    name=f"{self.Orb_ex_ind[i,0]}->{self.Orb_ex_ind[i,1]}",
                    line=dict(color=colors[i], width=2)),
                    row=1, col=1)   
      
      # add labels and titles
      fig.update_layout(
      title="Expansion coefficient as a function of time",
      xaxis_title="time (au)",
      yaxis_title=r"$|c_i(t)|^2$",
      legend=dict(
         orientation='h',
         yanchor="top",
         y=-0.2,
         #xanchor="left",
         #x=0.8,
         font=dict(size= 12)
         )
      )

      # add subplot for field      
      for i in range(self.V_list[0,:].shape[0]):
         fig.add_trace(go.Scatter(x=self.t_list, y=self.V_list[:,i],
                    mode='lines',
                    showlegend=False,
                    line=dict(color=color_field[i], width=2)),
                    row=2, col=2)
         fig.add_annotation(x=self.laser['t0'][i], y=self.laser["E0"][i],
            text="{:.2f} au".format(abs(self.laser['freq'][i])),
            font=dict(color=color_field[i]),
            row=2, col=2)
      
      return fig
        

## Analysis

In [22]:
# Initialize molecule
my_molecular_system = Build_basis()

# print dimension
print("Number of basis (Slater state) is ", my_molecular_system.S_tdm.shape)
print("Index of HOMO is ", my_molecular_system.HOMO_idx)
print("Transition energy from GS to first valence excited state is {:.2f} eV".format(
    abs(my_molecular_system.DE[my_molecular_system.HOMO_idx, my_molecular_system.HOMO_idx+1]*27.2114)))


Number of basis (Slater state) is  (78, 78, 3)
Index of HOMO is  17
Transition energy from GS to first valence excited state is 5.58 eV


In [44]:
print("Core-valence transition:")
print("Energy = {:.2f} eV".format(my_molecular_system.DE[0,18]*27.214))
print("TDM = ", my_molecular_system.S_tdm[0,18])
print()
print("Valence transition:")
print("Energy = {:.2f} eV".format(my_molecular_system.DE[18,20]*27.214))
print("TDM = ", my_molecular_system.S_tdm[18,20])


Core-valence transition:
Energy = -345.93 eV
TDM =  [0.        0.        0.0741941]

Valence transition:
Energy = -14.84 eV
TDM =  [0.06319119 0.         0.        ]


In [46]:
# Relevant transition frequencies
d1 = my_molecular_system.DE[0,18]
d2 = my_molecular_system.DE[18,19]

# Choose laser pulse

# Laser pulse parameters in a.u
E0 = [0.8, 0.8] # intensities
tau = [1.0, 1.0] #width
freq = [d1, d2]  # frequencies/energies of the field
t0 = [1.5, 5] # time at max(E0)
# integration parameters
tf = 10 # final time
laser = {'E0':E0,'tau':tau,'freq':freq,'t0':t0}

# Solve TDSE
my_molecular_system.Solve_TDSE(laser,tf)

# Plot results
#plt.rcParams["figure.figsize"]=8,6
f = my_molecular_system.Plot(0.001)
f.show()

There are 2000.0 integration points
This is the format of your plot grid:
⎡ (1,1) x,y   ⎤      (empty)    
⎣      :      ⎦  [ (2,2) x2,y2 ]

